# Two-box model estimating the effects of touristic activities on lakes

This script runs a two-box (epilimnion-hypolimnion) model of a lake to estimate the effects of touristic activities. 

**Author**: Tomy Doda, tomy.doda@unil.ch \
**Last update**: dd.mm.2025

### Load the packages

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'svg.fonttype':'none', 'font.sans-serif':'Arial','font.size': 12}) # "none": to export text as text
import pandas as pd
from datetime import datetime, timezone, UTC, timedelta
import math
import cmocean
from functions import load_parameters
# Uncomment to have interactive figures:
%matplotlib widget 

***

### Options (to change for different simulations)

Specify the lake name, according to the files in the `lakes` folder:

In [2]:
lakename="muzelle"

Specify the processes to include, according to the files in the `processes` folder:
* sunscreen

In [3]:
processes=["sunscreen"]

Specify the activities to include, according to the files in the `activities` folder:
* swimming

In [4]:
activities=["swimming"]

Figure saving option:

In [5]:
savefig=False # If True, all figures are exported as png

***

### Load the parameters from csv files

Lake-specific parameters:

In [6]:
param_lake,_,_=load_parameters(os.path.join("lakes",lakename+".csv"))

Process-specific parameters:

In [7]:
param_process=dict()
for p in processes:
    param_process[p],_,_=load_parameters(os.path.join("processes",p+".csv"))

Activity-specific parameters:

In [8]:
param_act=dict()
for a in activities:
    param_act[a],_,_=load_parameters(os.path.join("activities",a+".csv"))

In [9]:
param_act

{'swimming': {'Number_swimmers': 10.0,
  'Swimming_duration': 0.1,
  'Start_swimming_time': datetime.datetime(1900, 1, 1, 9, 0),
  'End_swimming_time': datetime.datetime(1900, 1, 1, 18, 0),
  'Start_swimming_day': datetime.datetime(2025, 7, 1, 0, 0),
  'End_swimming_day': datetime.datetime(2025, 8, 31, 0, 0)}}